In [22]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers.convolutional import Conv1D
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [54]:
import pandas as pd
data = pd.read_csv('Ola_Dataset1.csv')
data.head()
data.shape

(690, 6)

In [56]:
data=data[data.Analysis!="Neutral"]
data.shape

(459, 6)

In [57]:
data.drop('Subjectivity',axis=1,inplace=True)
data.drop('Polarity',axis=1,inplace=True)
data.drop('Analysis',axis=1,inplace=True)

In [58]:
data.reset_index(drop=True,inplace=True)

In [59]:
data.head()

,Unnamed: 0,tweet,label
0,0,not that it will matter to you no zamato swi...,1.0
1,1,Happy church day ⛪⛪ Marriott Hotel Downtown A...,1.0
2,2,Over drivers in London register on Ola Great ...,1.0
3,3,It would be great if you chain me in this ap...,1.0
4,4,Just wait Sir will sooner you face a welcome ...,1.0


In [60]:
import string
import re
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct
data['Text_Clean'] = data['tweet'].apply(lambda x: remove_punct(x))

In [61]:
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in data.Text_Clean]

In [62]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [63]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')
def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [removeStopWords(sen) for sen in lower_tokens]
data['Text_Final'] = [' '.join(sen) for sen in filtered_words]
data['tokens'] = filtered_words
data.head()
data.shape

(459, 6)

In [64]:
pos = []
neg = []
for l in data.label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)
print(data.shape)
data['Pos']= pos
data['Neg']= neg

data = data[['Text_Final', 'tokens', 'label', 'Pos', 'Neg']]
data.head()

(459, 6)


,Text_Final,tokens,label,Pos,Neg
0,matter zamato swiggy ola amazon flipkart agra ...,"[matter, zamato, swiggy, ola, amazon, flipkart...",1.0,1,0
1,happy church day ⛪⛪ marriott hotel downtown ab...,"[happy, church, day, ⛪⛪, marriott, hotel, down...",1.0,1,0
2,drivers london register ola great job httpstco...,"[drivers, london, register, ola, great, job, h...",1.0,1,0
3,would great chain appreciation,"[would, great, chain, appreciation]",1.0,1,0
4,wait sir sooner face welcome fire shouting hu,"[wait, sir, sooner, face, welcome, fire, shout...",1.0,1,0


In [65]:
data.head()

,Text_Final,tokens,label,Pos,Neg
0,matter zamato swiggy ola amazon flipkart agra ...,"[matter, zamato, swiggy, ola, amazon, flipkart...",1.0,1,0
1,happy church day ⛪⛪ marriott hotel downtown ab...,"[happy, church, day, ⛪⛪, marriott, hotel, down...",1.0,1,0
2,drivers london register ola great job httpstco...,"[drivers, london, register, ola, great, job, h...",1.0,1,0
3,would great chain appreciation,"[would, great, chain, appreciation]",1.0,1,0
4,wait sir sooner face welcome fire shouting hu,"[wait, sir, sooner, face, welcome, fire, shout...",1.0,1,0


In [67]:
data_train, data_test = train_test_split(data, 
                                         test_size=0.3, 
                                         random_state=78)

In [68]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

5487 words total, with a vocabulary size of 2037
Max sentence length is 35


In [69]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test['tokens']]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

2247 words total, with a vocabulary size of 1181
Max sentence length is 34


In [70]:
word2vec_path = 'C:/Users/hp/Sentiment-Analysis-RNN-LSTM/GoogleNews-vectors-negative300.bin'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [71]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [72]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train['Text_Final'].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train['Text_Final'].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
train_cnn_data = pad_sequences(training_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)

Found 2037 unique tokens.


In [73]:
train_embedding_weights = np.zeros((len(train_word_index)+1, 
 EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(2038, 300)


In [74]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
 
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    convs = []
    filter_sizes = [2,3,4,5,6]
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=10, 
                        kernel_size=filter_size, 
                        activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)
    l_merge = concatenate(convs, axis=1)
    x = Dropout(0.1)(l_merge)  
    x = Dense(75, activation='relu')(x)
    #x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [75]:
label_names = ['Pos', 'Neg']

In [76]:
model = ConvNet(train_embedding_weights, 
                MAX_SEQUENCE_LENGTH, 
                len(train_word_index)+1, 
                EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      611400      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 49, 10)       6010        embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 10)       9010        embedding[0][0]                  
_______________________________________________________________________________________

In [77]:
y_train = data_train[label_names].values

x_train = train_cnn_data
y_tr = y_train

In [81]:
num_epochs = 10
batch_size = 10
hist = model.fit(x_train, 
                 y_tr, 
                 epochs=num_epochs, 
                 validation_split=0.1, 
                 shuffle=True, 
                 batch_size=batch_size)

Epoch 1/10
29/29 [==============================] - 1s 27ms/step - loss: 0.2316 - acc: 0.9826 - val_loss: 0.4991 - val_acc: 0.7273
Epoch 2/10
29/29 [==============================] - 1s 19ms/step - loss: 0.1188 - acc: 0.9931 - val_loss: 0.4728 - val_acc: 0.7576
Epoch 3/10
29/29 [==============================] - 0s 17ms/step - loss: 0.0722 - acc: 0.9931 - val_loss: 0.4187 - val_acc: 0.8485
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0522 - acc: 0.9965 - val_loss: 0.4487 - val_acc: 0.7879
Epoch 5/10
29/29 [==============================] - 0s 17ms/step - loss: 0.0393 - acc: 1.0000 - val_loss: 0.4073 - val_acc: 0.7576
Epoch 6/10
29/29 [==============================] - 0s 17ms/step - loss: 0.0213 - acc: 1.0000 - val_loss: 0.4269 - val_acc: 0.8182
Epoch 7/10
29/29 [==============================] - 0s 17ms/step - loss: 0.0168 - acc: 1.0000 - val_loss: 0.4054 - val_acc: 0.7273
Epoch 8/10
29/29 [==============================] - 0s 16ms/step - loss: 0.0136 - a

In [82]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [83]:
predictions = model.predict(test_cnn_data, 
                            batch_size=3, 
                            verbose=1)
labels = [1, 0]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
sum(data_test.label==prediction_labels)/len(prediction_labels)

46/46 [==============================] - 0s 2ms/step


0.6231884057971014